<a href="https://colab.research.google.com/github/ahmadhuzaifa/Instagram-scraper/blob/master/Instagram_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instagram Public Post Scraper
Instagram-scraper is an application written in Python3 that scrapes and downloads instagram posts for puv Use responsibly.



##Necessary Libraries
This program was written in the language Python3 on Google Colab. Make sure the following are available for the program to run properly:
- Selenium (3.141.0+)
- chromium-chromedriver

In [18]:
!pip3 install selenium
!pip install selenium
!pip install chromedriver-autoinstaller
!copy /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
import time 
import selenium 

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
print("selenium Version = ", selenium.__version__)

The syntax of the command is incorrect.
selenium Version =  4.4.3


In [19]:
!pip install webdriver-manager

In [20]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\imvee\AppData\Local\Temp/ipykernel_14620/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [21]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
browser = webdriver.Chrome(ChromeDriverManager().install())
hashtag='food'
# browser.get('https://www.instagram.com/explore/tags/'+hashtag)
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

C:\Users\imvee\AppData\Local\Temp/ipykernel_14620/2162131179.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [22]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [23]:
class InstagramBot():
    def __init__(self):
      self.chrome_options = webdriver.ChromeOptions()
      self.browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      self.csvScrapedData = [["description", "weight", "location", "user", "time", "image"]]
      self.hashtag = ""
      
    def signIn(self, email, password):
      print("Signing In")
      self.email = email
      self.password = password
      self.browser.get('https://www.instagram.com/accounts/login/')
      emailInput = self.browser.find_elements_by_css_selector('form input')[0]
      passwordInput = self.browser.find_elements_by_css_selector('form input')[1]
      emailInput.send_keys(self.email)
      passwordInput.send_keys(self.password)
      passwordInput.send_keys(Keys.ENTER)
      time.sleep(2)
    
    def scrape(self, url):
      self.browser.get(url)
      self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      elements = self.browser.find_elements_by_xpath("//div[@class='v1Nh3 kIKUG  _bz0w']")
      hrefElements = self.browser.find_elements_by_xpath("//div[@class='v1Nh3 kIKUG  _bz0w']/a")
      elements_link = [x.get_attribute("href") for x in hrefElements]
      for elements in elements_link:
        self.scrapePost(elements)
 
    def scrapePost(self, url):
      self.browser.get(url)
      print("Scraping Post: " + url)
      try: 
        location_element = self.browser.find_element_by_xpath("//a[@class='O4GlU']").text
        location_element = location_element.replace(",", " ")
        user_element = self.browser.find_element_by_xpath("//a[@class='FPmhX notranslate nJAzx']")
        user_element_text = user_element.text
        user_element_text = user_element_text.replace(",", " ")
        user_element_link = user_element.get_attribute("href")
        try:
          desc_element = self.browser.find_element_by_xpath("//div[@class='C4VMK']/span")
          desc_text = desc_element.text
          desc_text = desc_text.replace("\n", " ")
          desc_text = desc_text.replace(",", " ")
        except:
          desc_text = " "
          pass
        try: 
          timestamp_element = self.browser.find_element_by_xpath("//div[@class='k_Q0X NnvRN']/a/time")
          timestamp = timestamp_element.get_attribute("datetime")
          timestamp = timestamp.replace("\n", " ")
          timestamp = timestamp.replace(",", " ")
        except:
          timestamp = " "
          pass    
        try:
          likes_element = self.browser.find_element_by_xpath("//a[@class='zV_Nj']/span").text
          likes_element = likes_element.replace(",", "")
          no_of_likes = int(likes_element)
          followerCount = self.findFollowerCount(user_element_link)
          weight = no_of_likes/followerCount
        except:
          weight = 0
          pass
        image_url = self.findImage()
        self.scrapedData = [desc_text, weight ,location_element, user_element_text, timestamp, image_url]
        print(self.scrapedData)
        self.csvScrapedData.append(self.scrapedData)
      except:
        pass
      
       
    def findImage(self):
      image_element = self.browser.find_element_by_xpath("//div[@class='KL4Bh']/img")
      image_element_link = image_element.get_attribute("src")
      return image_element_link

   
    def findFollowerCount(self, userURL):
      self.browser.get(userURL)
      followers_count_int = 0 
      try:
        count_element = self.browser.find_elements_by_xpath("//span[@class='g47SY ']")
        followers_count = count_element[1].get_attribute("title")
        followers_count = followers_count.replace(",", "")
        followers_count_int = int(followers_count)
      except:
          pass    
      return followers_count_int
    
    
    def scrapeWithHashtags(self, hashtags):
      for hashtag in hashtags:
        self.hashtag = hashtag
        print("-----------Scraping the hashtag '" + hashtag +"' -----------")
        url = 'https://www.instagram.com/explore/tags/' + hashtag
        self.scrape(url)
        
    def exportCSVFile(self):
      csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
      with open('csvScrapedData.csv', 'w') as f:
        writer = csv.writer(f, dialect='myDialect')
        for row in self.csvScrapedData:
          writer.writerow(row)
      f.close()
      


In [ ]:
scrape_type = input("Do you want to scrape any private posts [y/n]:\n")
hashtags = ['dog']
bot = InstagramBot()

if scrape_type == "y":
  username = input("What's your email?\n")
  password = input("What's your password?\n")
  bot.signIn(username, password)
elif scrape_type == "n":
  pass
else:
  sys.exit("No valid Input")

bot.scrapeWithHashtags(hashtags)
bot.exportCSVFile()
